In [11]:
import tensorflow as tf
from dataset import get_data
from model import get_model
import os
import numpy as np
import matplotlib
matplotlib.use('TKAgg')
import matplotlib.pyplot as plt



## Definiujemy sciezki

In [12]:
train_image_directory = os.path.join(os.getcwd(), "DATABASE/USGimage/train")
train_mask_directory = os.path.join(os.getcwd(), "DATABASE/USGimages_MASKS/train")

val_image_directory = os.path.join(os.getcwd(), "DATABASE/USGimage/valid")
val_mask_directory = os.path.join(os.getcwd(),"DATABASE/USGimages_MASKS/valid")

In [13]:
X_train, y_train = get_data(image_path=train_image_directory, mask_path=train_mask_directory)

print(X_train.shape, y_train.shape)
print(np.max(X_train), np.max(y_train))

(754, 256, 256, 3) (754, 256, 256, 1)
1.0 1.0


In [14]:
X_valid, y_valid = get_data(image_path=val_image_directory, mask_path=val_mask_directory)
print(X_valid.shape, y_valid.shape)
print(np.max(X_valid), np.max(y_valid))

(141, 256, 256, 3) (141, 256, 256, 1)
1.0 1.0


## Definicja modelu

In [15]:
img_size = (256,256)
num_classes = 1

model = get_model(img_size=img_size, num_classes=num_classes)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 256, 256, 16  448         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_9 (Dropout)            (None, 256, 256, 16  0           ['conv2d_19[0][0]']              
                                )                                                           

## Definicja treningu

In [16]:
for layer in model.layers:
    print(layer)

In [17]:
import segmentation_models as sm

In [18]:
batch_size = 16
epochs = 100
optimizer = tf.keras.optimizers.Adam(0.0001)
callbacks = [tf.keras.callbacks.ModelCheckpoint("./best_model.h5", save_best_only=True)]
metrics = [tf.keras.metrics.BinaryAccuracy(
    name='binary_accuracy', dtype=None, threshold=0.5
)]


In [19]:
model.compile(loss=sm.losses.bce_dice_loss, optimizer=optimizer, metrics = metrics)

In [20]:
training = model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid),
          batch_size=batch_size, epochs=epochs, callbacks=callbacks)

Epoch 1/100
48/48 [==============================] - 163s 3s/step - loss: 1.2042 - binary_accuracy: 0.8940 - val_loss: 1.1359 - val_binary_accuracy: 0.9376
Epoch 2/100
48/48 [==============================] - 161s 3s/step - loss: 0.9291 - binary_accuracy: 0.9143 - val_loss: 1.1777 - val_binary_accuracy: 0.9399
Epoch 3/100
48/48 [==============================] - 162s 3s/step - loss: 0.8365 - binary_accuracy: 0.9270 - val_loss: 0.9838 - val_binary_accuracy: 0.9252
Epoch 4/100
48/48 [==============================] - 161s 3s/step - loss: 0.7848 - binary_accuracy: 0.9320 - val_loss: 0.9667 - val_binary_accuracy: 0.9174
Epoch 5/100
48/48 [==============================] - 161s 3s/step - loss: 0.7434 - binary_accuracy: 0.9341 - val_loss: 0.9844 - val_binary_accuracy: 0.9462
Epoch 6/100
48/48 [==============================] - 161s 3s/step - loss: 0.7662 - binary_accuracy: 0.9351 - val_loss: 0.9571 - val_binary_accuracy: 0.9063
Epoch 7/100
48/48 [==============================] - 161s 3s/ste

In [21]:
plt.plot(training.history['binary_accuracy'])
plt.plot(training.history['val_binary_accuracy'])
plt.title('Model Binary Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(training.history['loss'])
plt.plot(training.history['val_loss'])
plt.title('Dice Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [22]:
test_image_directory = os.path.join(os.getcwd(), "DATABASE/USGimage/test")
test_mask_directory = os.path.join(os.getcwd(), "DATABASE/USGimages_MASKS/test")


In [23]:
X_test, y_test = get_data(image_path=test_image_directory, mask_path=test_mask_directory)

print(X_test.shape, y_test.shape)
print(np.max(X_test), np.max(y_test))

(48, 256, 256, 3) (48, 256, 256, 1)
1.0 1.0


In [24]:
model.load_weights('best_model.h5')

test_preds = model.predict(X_test)

2/2 [==============================] - 2s 616ms/step


In [25]:
preds_test_thresh = (test_preds >= 0.5).astype(np.uint8)

In [35]:
test_img = preds_test_thresh[47, :, :, 0]

figure,axis = plt.subplots(1,2)
axis[0].imshow(test_img, cmap='gray')
axis[0].set_title(" Predicted image")

axis[1].imshow(y_test[47], cmap='gray')
axis[1].set_title("Original mask")
plt.show()

In [38]:
def dice_similarity(X_test, y_test):
    # calculate the Dice similarity coefficient
    if np.max(X_test) == 0 and np.max(y_test) == 0:
        return 1
    intersection = tf.reduce_sum(y_test * X_test)
    return 2 * intersection / (tf.reduce_sum(X_test) + tf.reduce_sum(y_test))

dsc = np.zeros(test_preds.shape[0])


for i in range(test_preds.shape[0]):
    dsc[i]=dice_similarity(test_preds[i,:,:,:],y_test[i,:,:,:])
with open(f"./results_dice+dice_loss.txt",'w') as f:
    for i in range(test_preds.shape[0]):
        f.write(f"{dsc[i]}\n")

    
